# Select Your IPTS 

In [63]:
from __code import system
system.System.select_working_dir()

** Using Debugging Mode! **


HBox(children=(Label(value='Select Working Folder', layout=Layout(width='20%')), Select(index=11, options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-16259', 'IPTS_13639_resonance_shawn', 'IPTS_14832', 'IPTS_16212', 'IPTS_16475', 'IPTS_16893', 'IPTS_17099', 'IPTS_17685', 'IPTS_17981', 'IPTS_18806', 'IPTS_19037', 'IPTS_19558', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'iBeatles_debugging'), value='IPTS_17099')))

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - selection of a **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [64]:
from __code.__all import custom_style
custom_style.style()

In [65]:
from __code.normalization import NormalizationHandler

In [66]:
%matplotlib notebook

In [67]:
from ipywidgets import widgets, Layout
from IPython.core.display import display
import ipywe.fileselector


In [68]:
def close(w):
    "recursively close a widget"
    if hasattr(w, 'children'):
        for c in w.children:
            close(c)
            continue
    w.close()
    return

class Panel:
    
    layout = Layout(border='1px lightgray solid', margin='5px', padding='15px')
    button_layout = Layout(margin='10px 5px 5px 5px')
    label_layout = Layout(height='32px', padding='2px', width='300px')
    
    def show(self):
        display(self.top_panel)
        display(self.bottom_panel)
        self.file_selector.show()
        
    def remove(self):
        close(self.top_panel)
        close(self.bottom_panel)
        self.file_selector.remove()
        
    def nextStep(self):
        raise NotImplementedError
        

In [69]:
class WizardPanel:
    
    label_layout = Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px')
    
    def __init__(self, start_panel):
        display(widgets.Label("STEP 1: Selection of Input Files",
                             layout=self.label_layout))
        start_panel.show()
        return
        

In [80]:
class myFileSelectorPanel(ipywe.fileselector.FileSelectorPanel):
    
    def __init__(self, instruction, start_dir=".", type='file', 
                 next=None, multiple=False, 
                 newdir_toolbar_button=False, 
                 current_ui=None
                 parent_ui=None):
        super(myFileSelectorPanel, self).__init__(instruction, 
                                                  start_dir= start_dir, 
                                                  type=type, 
                                                  next=next, 
                                                  multiple=multiple,
                                                  newdir_toolbar_button=newdir_toolbar_button)
        self.current_ui = current_ui
        self.parent_ui = parent_ui
    
    def validate(self, s):
        self.current_ui.status_label.value = "{}".format(len(self.parent_ui.sample_files))
        super(myFileSelectorPanel, self).validate(s)
        

In [81]:
class SampleSelectionPanel(Panel):

    sample_files = []
    ob_files = []
    df_files = []

    def __init__(self):
        
        # top panel
        sample_title_ui = widgets.HBox([widgets.Label("Instructions:",
                                                     layout = widgets.Layout(width='20%')),
                                       widgets.Label("Select Samples Images and click NEXT",
                                                    layout = widgets.Layout(width='50%'))])

        sample_label_ui = widgets.HBox([widgets.Label("Sample selected:",
                                                     layout = widgets.Layout(width='20%')),
                                       widgets.Label("None",
                                                    layout = widgets.Layout(width='50%')),
                                       ])
        self.status_label = sample_label_ui.children[1]
        
        self.top_panel = widgets.VBox(children=[sample_title_ui, sample_label_ui],
                                 layout=self.layout)
        
        # bottom panel
        self.bottom_panel = widgets.HBox([widgets.Label("         ",
                                             layout=widgets.Layout(width='70%')),
                               widgets.Button(description = "Next (Open Beam)  >>",
                                              tooltip = 'Click to move to next step',
                                              button_style = 'info',
                                             layout=widgets.Layout(width='20%'))])
        
        # file selector        
        self.file_selector = myFileSelectorPanel(instruction='',
                                                 start_dir = system.System.get_working_dir(),
                                                 multiple = True,
                                                 current_ui = self,
                                                 parent_ui = self)
#         self.file_selector = ipywe.fileselector.FileSelectorPanel(instruction='',
#                                                              start_dir = system.System.get_working_dir(),
#                                                             multiple = True)

        next_button = self.bottom_panel.children[1]
        next_button.on_click(self.next_panel)

    def next_panel(self, event):
        try:
            self.sample_files = self.file_selector.selected
        except:
            self.sample_files = []
        
        self.remove()
        _panel = OBSelectionPanel(top_parent=self)
        _panel.show()

        

In [82]:
class OBSelectionPanel(Panel):
    
    def __init__(self, top_parent=None):
        
        self.parent = top_parent
        
        # top panel
        sample_title_ui = widgets.HBox([widgets.Label("Instructions:",
                                                     layout = widgets.Layout(width='20%')),
                                       widgets.Label("Select OB Images and click NEXT",
                                                    layout = widgets.Layout(width='50%'))])

        nbr_ob = len(self.parent.ob_files)
        if nbr_ob > 0:
            ob_selected = "{}".format(nbr_ob)
        else:
            ob_selected = None
        
        sample_label_ui = widgets.HBox([widgets.Label("OB selected:",
                                                     layout = widgets.Layout(width='20%')),
                                       widgets.Label(ob_selected,
                                                    layout = widgets.Layout(width='50%')),
                                       ])
        self.status_label = sample_label_ui.children[1]
        
        self.top_panel = widgets.VBox(children=[sample_title_ui, sample_label_ui],
                                 layout=self.layout)

        
        # bottom panel
        self.bottom_panel = widgets.HBox([widgets.Button(description = "<< Previous (Sample)",
                                                  tooltip = 'Click to move to previous step',
                                                  button_style = 'info',
                                                 layout=widgets.Layout(width='20%')),
                                          widgets.Label("         ",
                                                 layout=widgets.Layout(width='70%')),
                                          widgets.Button(description = "Next (Dark Field)  >>",
                                                  tooltip = 'Click to move to next step',
                                                  button_style = 'info',
                                                 layout=widgets.Layout(width='20%'))])
        
        prev_button = self.bottom_panel.children[0]
        prev_button.on_click(self.prev_panel)
        
        next_button = self.bottom_panel.children[2]
        next_button.on_click(self.next_panel)

        # file selector
        self.file_selector = myFileSelectorPanel(instruction='',
                                                 start_dir = system.System.get_working_dir(),
                                                 multiple = True)
#         self.file_selector = ipywe.fileselector.FileSelectorPanel(instruction='',
#                                                              start_dir = system.System.get_working_dir(),
#                                                             multiple = True)

    def next_panel(self, event):
        try:
            self.parent.ob_files = self.file_selector.selected
        except:
            self.parent.ob_files = []
        self.remove()
        _panel = DFSelectionPanel(top_parent=self.parent)
        _panel.show()
        
    def prev_panel(self, event):
        self.remove()
        _panel = SampleSelectionPanel(top_parent=self.parent)
        _panel.show()


In [83]:
class DFSelectionPanel(Panel):
    
    def __init__(self, top_parent=None):
        
        self.parent = top_parent
        
        # top panel
        sample_title_ui = widgets.HBox([widgets.Label("Instructions:",
                                                     layout = widgets.Layout(width='20%')),
                                       widgets.Label("Select DF Images and/or click NEXT",
                                                    layout = widgets.Layout(width='50%'))])

        sample_label_ui = widgets.HBox([widgets.Label("DF selected:",
                                                     layout = widgets.Layout(width='20%')),
                                       widgets.Label("None",
                                                    layout = widgets.Layout(width='50%')),
                                       ])
        self.status_label = sample_label_ui.children[1]

        self.top_panel = widgets.VBox(children=[sample_title_ui, sample_label_ui],
                                 layout=self.layout)
        
        # bottom panel
        self.bottom_panel = widgets.HBox([widgets.Button(description = "<< Previous (OB)",
                                                  tooltip = 'Click to move to previous step',
                                                  button_style = 'info',
                                                 layout=widgets.Layout(width='20%')),
                                          widgets.Label("         ",
                                                 layout=widgets.Layout(width='70%')),
                                          widgets.Button(description = "Next (Loading)  >>",
                                                  tooltip = 'Click to move to next step',
                                                  button_style = 'info',
                                                 layout=widgets.Layout(width='20%'))])
        
        prev_button = self.bottom_panel.children[0]
        prev_button.on_click(self.prev_panel)
        
        next_button = self.bottom_panel.children[2]
        next_button.on_click(self.next_panel)

        # file selector
        self.file_selector = ipywe.fileselector.FileSelectorPanel(instruction='',
                                                             start_dir = system.System.get_working_dir(),
                                                            multiple = True)

    def next_panel(self, event):
        try:
            self.parent.df_files = self.file_selector.selected
        except:
            self.parent.df_files = []
            
        self.remove()
        # loading
        
    def prev_panel(self, event):
        self.remove()
        _panel = OBSelectionPanel(top_parent=self.parent)
        _panel.show()

In [84]:
sample_panel = SampleSelectionPanel()
wizard = WizardPanel(sample_panel)

HTML(value='Please wait...')

Label(value='STEP 1: Selection of Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select Samples Images and click NEXT', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='None', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='info', description='Next (Open Beam)  >>', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_17099', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                          |     Fri Oct 27 10:11:51 2017', ' cylindrical_geometry_corrected     |     Directory', ' left_edge                          |     Directory', ' normalized_cropped                 |     Directory', ' ob                                 |     Directory', ' radiographs                        |     Directory', ' right_edge                         |     Directory', ' testing_new_output_format          |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_17099/left_edge', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' 20170113_LLZO_cycling_0120_0000.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0001.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0002.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0003.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0004.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0005.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0006.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0007.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0008.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0009.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0010.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0011.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0012.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0013.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0014.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0015.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0016.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0017.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0018.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0019.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0020.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0021.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0022.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0023.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0024.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0025.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0026.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0027.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0028.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0029.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0030.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0031.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0032.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0033.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0034.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0035.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0036.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0037.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0038.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0039.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0040.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0041.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0042.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0043.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_LLZO_cycling_0120_0044.tiff     |     Wed Feb  1 17:08:38 2017', ' 20170113_L

In [85]:
def sample_image():
    sample_title = widgets.Label("Select SAMPLES IMAGES and click NEXT")
    display(sample_title)

    sample_label_ui = widgets.HBox([widgets.Label("Current sample images selected:",
                                                 layout = widgets.Layout(width='20%')),
                                   widgets.Label("None",
                                                layout = widgets.Layout(width='50%')),
                                   ])
    display(sample_label_ui)
    title_ui = sample_label_ui.children[0]

    select_images = ipywe.fileselector.FileSelectorPanel(instruction='',
                                                         start_dir = system.System.get_working_dir(),
                                                        multiple = True)
    select_images.show()

    next_ui = widgets.HBox([widgets.Label("         ",
                                         layout=widgets.Layout(width='70%')),
                           widgets.Button(description = "Next   >>",
                                          tooltip = 'Click to move to next step',
                                          button_style = 'info',
                                         layout=widgets.Layout(width='20%'))])
    display(next_ui)
    
    next_button = next_ui.children[1]
    next_button.on_click(ob_images)
    
def ob_images(event):
    ob_title = widgets.Label("Select OB IMAGES and click NEXT")
    display(ob_title)

    ob_label_ui = widgets.HBox([widgets.Label("Current ob images selected:",
                                                 layout = widgets.Layout(width='20%')),
                                   widgets.Label("None",
                                                layout = widgets.Layout(width='50%')),
                                   ])
    display(ob_label_ui)
    title_ui = ob_label_ui.children[0]

    select_images = ipywe.fileselector.FileSelectorPanel(instruction = '',
                                                         start_dir = system.System.get_working_dir(),
                                                        multiple = True)
    select_images.show()

    prev_next_ui = widgets.HBox([widgets.Button(description= "<< Previous",
                                          tooltip = 'Click to move to previous step',
                                          button_style = 'info',
                                          layout = widgets.Layout(width='20%')),
                            widgets.Label("         ",
                                         layout=widgets.Layout(width='70%')),
                           widgets.Button(description = "Next   >>",
                                          tooltip = 'Click to move to next step',
                                          button_style = 'info',
                                         layout=widgets.Layout(width='20%'))])
    display(prev_next_ui)
    
sample_image()


Label(value='Select SAMPLES IMAGES and click NEXT')

HBox(children=(Label(value='Current sample images selected:', layout=Layout(width='20%')), Label(value='None', layout=Layout(width='50%'))))

HTML(value='Please wait...')

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_17099', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                          |     Fri Oct 27 10:11:51 2017', ' cylindrical_geometry_corrected     |     Directory', ' left_edge                          |     Directory', ' normalized_cropped                 |     Directory', ' ob                                 |     Directory', ' radiographs                        |     Directory', ' right_edge                         |     Directory', ' testing_new_output_format          |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='info', description='Next   >>', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

# Select Sample Images 

In [ ]:
working_dir = system.System.get_working_dir())
o_norm_handler = NormalizationHandler(working_dir = working_dir)
o_norm_handler.select_images()

# Select Open Beam (OB)

In [ ]:
o_norm_handler.select_images(data_type='ob')

# Select Dark Field (DF) - Optional

In [ ]:
o_norm_handler.select_images(data_type='df')

# Loding data 

In [ ]:
o_norm_handler.load_data()

# Preview Data

In [ ]:
o_norm_handler.plot_images(data_type='sample')

In [ ]:
o_norm_handler.plot_images(data_type='ob')

# Select Background Region 

In [ ]:
o_norm_handler.with_or_without_roi()

In [ ]:
o_norm_handler.select_sample_roi()

# Normalization

In [ ]:
o_norm_handler.run_normalization()

# Export 

In [ ]:
o_norm_handler.select_export_folder()

In [ ]:
o_norm_handler.export()